In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import get_dummies from google.colab import files


In [3]:
from google.colab import files
uploaded = files.upload()
uploaded1 = files.upload()

Saving Data_Train.xlsx to Data_Train.xlsx


Saving Data_Test.xlsx to Data_Test.xlsx


In [0]:
import io
dfTest = pd.read_excel(io.BytesIO(uploaded1['Data_Test.xlsx'])) #Loading and viewing the test dataset
dfTrain = pd.read_excel(io.BytesIO(uploaded['Data_Train.xlsx'])) #Loading and viewing the train dataset

In [11]:
dfTest.dropna(inplace = True)
dfTrain.dropna(inplace = True)
dfTest['Price'] = 1e8
print(dfTest.head(), dfTrain.head())

                                        Name    Location  ...  Seats        Price
0                    Maruti Alto K10 LXI CNG       Delhi  ...    4.0  100000000.0
1              Maruti Alto 800 2016-2019 LXI  Coimbatore  ...    5.0  100000000.0
2  Toyota Innova Crysta Touring Sport 2.4 MT      Mumbai  ...    7.0  100000000.0
3                       Toyota Etios Liva GD   Hyderabad  ...    5.0  100000000.0
4                          Hyundai i20 Magna      Mumbai  ...    5.0  100000000.0

[5 rows x 12 columns]                                Name    Location  Year  ...      Power Seats  Price
0            Maruti Wagon R LXI CNG      Mumbai  2010  ...  58.16 bhp   5.0   1.75
1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015  ...  126.2 bhp   5.0  12.50
2                      Honda Jazz V     Chennai  2011  ...   88.7 bhp   5.0   4.50
3                 Maruti Ertiga VDI     Chennai  2012  ...  88.76 bhp   7.0   6.00
4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013  ...  140.8 bhp 

In [27]:
frames = [dfTrain, dfTest]
dfCombined = pd.concat(frames)
dfCombined.reset_index(drop = True, inplace = True)
dfCombined.shape

(7198, 12)

In [28]:
print(dfCombined.head())
print(dfCombined.tail())

                               Name    Location  Year  ...      Power Seats  Price
0            Maruti Wagon R LXI CNG      Mumbai  2010  ...  58.16 bhp   5.0   1.75
1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015  ...  126.2 bhp   5.0  12.50
2                      Honda Jazz V     Chennai  2011  ...   88.7 bhp   5.0   4.50
3                 Maruti Ertiga VDI     Chennai  2012  ...  88.76 bhp   7.0   6.00
4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013  ...  140.8 bhp   5.0  17.74

[5 rows x 12 columns]
                                                   Name  ...        Price
7193                  Volkswagen Vento Diesel Trendline  ...  100000000.0
7194                             Volkswagen Polo GT TSI  ...  100000000.0
7195                             Nissan Micra Diesel XV  ...  100000000.0
7196                             Volkswagen Polo GT TSI  ...  100000000.0
7197  Mercedes-Benz E-Class 2009-2013 E 220 CDI Avan...  ...  100000000.0

[5 rows x 12 columns]


In [0]:
dfCombined['Mileage'] = dfCombined['Mileage'].apply(lambda x: float(list(x.split(' '))[0]))
dfCombined['Engine'] = dfCombined['Engine'].apply(lambda x: float(list(x.split(' '))[0]))  #Cleaning the data a little bit
dfCombined['Power'] = dfCombined['Power'].apply(lambda x: list(x.split(' '))[0]) #from experience, power has a 'null' string entry
lisNull = [index for index, element in enumerate(dfCombined['Power']) if element == 'null']
dfCombined.drop(lisNull, inplace = True)
dfCombined['Power'] = dfCombined['Power'].astype('float64')

In [30]:
dfCombined.head()
dfCombined.shape

(7073, 12)

In [31]:
#Vectorising the text data, lemmatisation and other advanced text normalisation wasn't done because of obvious reasons

textData = dfCombined['Name']
from sklearn.feature_extraction.text import TfidfVectorizer
vectoriser = TfidfVectorizer()
textData = vectoriser.fit_transform(textData)

from sklearn.decomposition import TruncatedSVD

svDec = TruncatedSVD(n_components=5, n_iter=10)

textDataTransformed = svDec.fit_transform(textData)
textDataTransformed.shape

dfCombined.drop(['Name'], axis = 1, inplace = True)

dfCombined = pd.concat([dfCombined, pd.DataFrame(textDataTransformed)], axis = 1)
dfCombined.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,0,1,2,3,4
0,Mumbai,2010.0,72000.0,CNG,Manual,First,26.60,998.0,58.16,5.0,1.75,0.032974,0.360398,-0.008919,0.006582,-0.021047
1,Pune,2015.0,41000.0,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50,0.483737,-0.039499,-0.000391,0.029244,0.062713
2,Chennai,2011.0,46000.0,Petrol,Manual,First,18.20,1199.0,88.70,5.0,4.50,0.018396,0.006070,0.395627,-0.058941,0.019167
3,Chennai,2012.0,87000.0,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00,0.042067,0.476564,-0.015554,-0.018619,0.006441
4,Coimbatore,2013.0,40670.0,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74,0.006327,0.002710,0.010114,0.114445,0.211779


In [0]:
mEnc = pd.get_dummies(dfCombined, columns = ['Location', 'Fuel_Type', 'Transmission', 'Owner_Type'])

In [33]:
mEnc.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,0,1,2,3,4,Location_Ahmedabad,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_LPG,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third
0,2010.0,72000.0,26.60,998.0,58.16,5.0,1.75,0.032974,0.360398,-0.008919,0.006582,-0.021047,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0
1,2015.0,41000.0,19.67,1582.0,126.20,5.0,12.50,0.483737,-0.039499,-0.000391,0.029244,0.062713,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
2,2011.0,46000.0,18.20,1199.0,88.70,5.0,4.50,0.018396,0.006070,0.395627,-0.058941,0.019167,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
3,2012.0,87000.0,20.77,1248.0,88.76,7.0,6.00,0.042067,0.476564,-0.015554,-0.018619,0.006441,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0
4,2013.0,40670.0,15.20,1968.0,140.80,5.0,17.74,0.006327,0.002710,0.010114,0.114445,0.211779,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0


In [37]:
mEnc.shape

(7197, 33)

In [0]:
mEnc.dropna(inplace = True)
mEnc.reset_index(drop = True, inplace = True)

In [43]:
from scipy import stats
zScore = np.abs(stats.zscore(mEnc['Kilometers_Driven']))
R = mEnc[zScore < 3]
R.info()
zScore = np.abs(stats.zscore(R['Mileage']))
R = R[zScore < 3]
# R.info()
zScore = np.abs(stats.zscore(R['Engine']))
R = R[zScore < 3]
# R.info()
zScore = np.abs(stats.zscore(R['Power']))
R = R[zScore < 3]
R.fillna(1e8, inplace = True)
RN = R[R['Price']==1e8]
RP = R[R['Price']!= 1e8]
print(RN.shape, RP.shape)
RN.reset_index(drop = True, inplace = True)
RP.reset_index(drop=True, inplace = True)

zScore = np.abs(stats.zscore(RP['Price']))
RP = RP[zScore < 3]

framesNew = [RN, RP]
rEnc = pd.concat(framesNew)
rEnc.reset_index(drop=True, inplace = True)
rEnc.info()

rEncY = rEnc.iloc[:, 6]
rEnc.drop(['Price'], axis = 1, inplace= True)
rEnc.head()
rEnc = rEnc.apply(lambda x: (x-np.mean(x))/(np.std(x)))
rEnc.head()

rEnc['Price'] = rEncY
rEnc.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 6941 entries, 0 to 6948
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       6941 non-null   float64
 1   Kilometers_Driven          6941 non-null   float64
 2   Mileage                    6941 non-null   float64
 3   Engine                     6941 non-null   float64
 4   Power                      6941 non-null   float64
 5   Seats                      6941 non-null   float64
 6   Price                      6941 non-null   float64
 7   0                          6941 non-null   float64
 8   1                          6941 non-null   float64
 9   2                          6941 non-null   float64
 10  3                          6941 non-null   float64
 11  4                          6941 non-null   float64
 12  Location_Ahmedabad         6941 non-null   uint8  
 13  Location_Bangalore         6941 non-null   uint8

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,0,1,2,3,4,Location_Ahmedabad,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_LPG,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third,Price
0,-0.136709,-0.093197,1.525683,-1.489205,-1.410520,-0.372011,-0.424614,-0.379656,2.916587,-1.103199,0.109309,-0.203464,-0.246508,-0.298719,2.974095,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,-0.390914,-0.344531,-0.082754,-1.051825,-0.042627,1.070171,-0.583866,0.583866,-2.158596,-0.036908,2.314180,-0.135337,100000000.0
1,1.134503,-0.698360,-1.189595,1.600497,0.980964,2.135603,0.032790,-0.434952,1.944849,-0.727607,0.284909,-0.203464,-0.246508,-0.298719,-0.336237,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,2.558105,-0.344531,-0.082754,0.950729,-0.042627,-0.934430,-0.583866,0.583866,0.463264,-0.036908,-0.432119,-0.135337,100000000.0
2,0.181094,-0.846012,-0.001969,-0.713394,-0.564577,-0.372011,-0.531676,-0.424485,-0.341221,-0.300156,0.158203,-0.203464,-0.246508,-0.298719,-0.336237,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,2.558105,-0.344531,-0.082754,-1.051825,-0.042627,1.070171,-0.583866,0.583866,0.463264,-0.036908,-0.432119,-0.135337,100000000.0
3,0.816700,0.825667,-0.617958,1.186473,0.795356,2.135603,-0.409555,-0.355374,2.977251,-0.830703,-0.091481,-0.203464,-0.246508,-0.298719,2.974095,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,-0.390914,-0.344531,-0.082754,0.950729,-0.042627,-0.934430,-0.583866,0.583866,-2.158596,-0.036908,2.314180,-0.135337,100000000.0
4,0.498897,0.039896,-1.468022,2.740030,1.478298,2.135603,-0.535485,-0.416006,-0.304514,-0.145867,0.887705,-0.203464,-0.246508,-0.298719,-0.336237,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,-0.390914,2.902499,-0.082754,0.950729,-0.042627,-0.934430,1.712721,-1.712721,0.463264,-0.036908,-0.432119,-0.135337,100000000.0


In [0]:
rEnc.to_csv('combibedSetNormalisedZScoreDimensionReduced.csv')

In [45]:
from google.colab import drive
drive.mount('drive', force_remount=True)
!cp combibedSetNormalisedZScoreDimensionReduced.csv "drive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive
